<a href="https://colab.research.google.com/github/kencav/project-lion/blob/master/notebooks/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:



*  Modularize data generation
    * Structure data-gen class
*  Expand dataset selection
    * Enron dataset
    * Discord scraper
    * Stanford paper (add link to this)



In [ ]:
# INSTALL GENERAL DEPENDENCIES

#! python3
!pip install openai
# Downgrade pandas to be compatible with Colab
!pip install pandas==1.1.0

#libraries
import os
import openai

import datetime



  Using cached pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.0
    Uninstalling pandas-1.1.0:
      Successfully uninstalled pandas-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


  Using cached pandas-1.1.0-cp37-cp37m-manylinux1_x86_64.whl (10.5 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai 0.11.5 requires pandas>=1.2.3, but you have pandas 1.1.0 which is incompatible.


Mounted at /content/drive


In [ ]:
# CONNECT KAGGLE

from google.colab import drive
drive.mount('/content/drive')

!pip install -q kaggle
!pip install -q kaggle-cli
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/
!cat ~/.kaggle/kaggle.json 
!chmod 600 ~/.kaggle/kaggle.json

# Format of kaggle download command (non-competition datasets). ENRON EMAILS DATASET CRASHES JESSIE'S LAPTOP
#!kaggle datasets download -d wcukierski/enron-email-dataset -p enron_emails

# SAMPLE CODE; DESIRED DATASET TOO BIG

# Alternate way to download kaggle files
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
# Download all
api.dataset_download_files('imdevskp/corona-virus-report')

# Download 1
api.dataset_download_file('imdevskp/corona-virus-report','covid_19_clean_complete.csv')

zf = ZipFile('covid_19_clean_complete.csv.zip')
#extracted data is saved in the same directory as notebook
zf.extractall() 
zf.close()

import pandas as pd
data=pd.read_csv('covid_19_clean_complete.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  ERROR: Failed building wheel for lxml
    Running setup.py install for lxml ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-imsnh6e8/lxml_36195e738bda45629f305335db51ffa6/setup.py'"'"'; __file__='"'"'/tmp/pip-install-imsnh6e8/lxml_36195e738bda45629f305335db51ffa6/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-vz0cg00a/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/lxml Check the logs for full command output.
{"username":"apersonof

In [ ]:
# CONNECT TWITTER 

#! python3

#libraries
import pandas as pd
import tweepy
import csv
import os
import datetime

#Twitter API credentials
consumer_key = 'CcoYGfKzL7Hgmk4d2K0F8p7ff'
consumer_secret = 'lBGQo8qYEY8UxPxljSASvEbY6BwTycCAstmQgVkTqmNF8iGtbK'
access_key = '4600137193-q3fORQZDy4KproA7hv4hTGtZ6PjLF7fJUboH20x'
access_secret = 'jBQMNfZq9WkSQT2O8kAV79f9BKtuPtYc5WlpQRhe5WWK4'


API_KEY = "Y0iyPQIvdeAAUWgzvIJGFOApS"



def validate_twitter(consumer_key, consumer_secret, access_key, access_secret):
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_key, access_secret)

  api = tweepy.API(auth)

  return api

def sort_tweets_by_time_chunk(input_df, column_name='Date', time_chunk="week"):
    for x in range(52):
      df = input_df[input_df[column_name].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{screen_name}.csv')

# get an arbitrary # of tweets
def get_tweets(screen_name, num, api):
    new_tweets = api.user_timeline(screen_name=screen_name, count=num)
    
    return new_tweets


def get_all_tweets(screen_name, api):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name=screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name,
                                       count=200,
                                       max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print(f"...{len(alltweets)} tweets downloaded so far")

    #transform the tweepy tweets into a 2D array that will populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text]
                 for tweet in alltweets]
                 
    # Transposed in the repl.it
    tweets_df = pd.DataFrame(outtweets,
                             columns=["Tweet_id", "Date", "Tweet_Text"])
    
    # TODO: incorporate this start to error handling
    # except BaseException as e:
    #       print('failed on_status', str(e))
    #       time.sleep(3)

  # For each week in the year, create dir if ! exists, filter df, write to csv, and save to folder
  # See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekofyear.html  
    '''
    for x in range(52):
      df = tweets_df[tweets_df['Date'].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{screen_name}.csv')
    '''
    return tweets_df

    
# If using datetime object, can use this function to get the week for an individual tweet
# Note that every python data type handles this slightly differently! Check the type!
def make_chunks(data, tuple_position=1):
  # 0 = year, 1 = week, 2 = day
  weeks = []
  for x in data:
    chunk = x.created_at.isocalendar()[tuple_position]
    weeks.append(chunk)
  return weeks

# #tweets
# tweets = pd.read_csv('bored_tweets.csv')

# #remove links
# no_links = tweets[~tweets.text.str.contains("http")]
# no_links.head()

# # only tweets
# just_tweets = no_links.text

# # make everything a string
# text = str(just_tweets)


api = validate_twitter(consumer_key, consumer_secret, access_key, access_secret)
test = get_all_tweets('BoredElonMusk', api)
print(test)


getting tweets before 1476076237432901632
...400 tweets downloaded so far
getting tweets before 1472332160543559681
...600 tweets downloaded so far
getting tweets before 1468350290092908544
...800 tweets downloaded so far
getting tweets before 1461187944123748357
...1000 tweets downloaded so far
getting tweets before 1456288787831537665
...1200 tweets downloaded so far
getting tweets before 1453461088251506692
...1400 tweets downloaded so far
getting tweets before 1445819076073328644
...1600 tweets downloaded so far
getting tweets before 1442996049262493697
...1800 tweets downloaded so far
getting tweets before 1439633102150635522
...2000 tweets downloaded so far
getting tweets before 1436130768128917531
...2200 tweets downloaded so far
getting tweets before 1433763634119729154
...2400 tweets downloaded so far
getting tweets before 1430462321915088896
...2600 tweets downloaded so far
getting tweets before 1427079018990374914
...2800 tweets downloaded so far
getting tweets before 142084

In [ ]:
# CONNECT DISCORD #

import requests

headers = {"Authorization": f"Bot {bot_token}"}

test_channel_id = 885948250253848588
test_guild_id = 647140436686667776

channel_path = f'https://discord.com/api/channels/{test_channel_id}/messages'

guild_path_channels = f'https://discord.com/api/guilds/{test_guild_id}/channels'


res = requests.get(channel_path, headers=headers).json()

for x in res:
  print(x)



In [ ]:
# SUPPORT FUNCTIONS #
import sys
import shutil

# Lazy util function
def check_type(data):
  print(type(data))

def NUKE_ALL_FOLDERS_AND_FILES():
  print("Nuclear launch detected")
  # Get directory name
  # TODO: Update this to be more flexible, but it works for now
  for x in range(1,52):
    try:
        shutil.rmtree(f'week-{x}')
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))

#NUKE_ALL_FOLDERS_AND_FILES()

#os.listdir()

In [ ]:


# TODO: Objectify functions

def generate_df(input_df, file_path_name):
  '''
  - For x in df, grab x['date'] and split into day/month/year/week of year
  - Make columns for each val and assign accordingly
  - This will result in one large spreadsheet that you can filter
  '''
  pass

def parse_by_weeks(input_df, file_path_name):
    for x in range(52):
      df = input_df[input_df['Date'].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{file_path_name}.csv')
    return input_df

  
generate_data(test, "test")

<bound method DataFrame.count of Empty DataFrame
Columns: [Tweet_id, Date, Tweet_Text]
Index: []>
<bound method DataFrame.count of                Tweet_id  ...                                         Tweet_Text
0   1478508067213885440  ...  @jack A lot of energy is spent (wasted) debati...
1   1478504331057852417  ...                               @griillz_eth I'm in.
2   1478502749281001472  ...  @mattmireles @john_c_palmer @makinmarkets Ange...
3   1478455370481766400  ...  @jamie247 I'd like to learn more but first nee...
4   1478445682075729923  ...  @brianjcho @amytongwu Cosign on those and if y...
..                  ...  ...                                                ...
70  1477839129744314371  ...  @cobie @EFillae @CryptOH10 @Comyk34 @LeCryptog...
71  1477824055835168774  ...  @bchesky @levie I’m still salty Aaron’s compan...
72  1477817531159101444  ...                                @austin_rief Savage
73  1477817222500274176  ...  @TheChainGamer @GuildOfGuardian @AxieIn

,Tweet_id,Date,Tweet_Text
0,1478508067213885440,2022-01-04 23:26:07,@jack A lot of energy is spent (wasted) debati...
1,1478504331057852417,2022-01-04 23:11:16,@griillz_eth I'm in.
2,1478502749281001472,2022-01-04 23:04:59,@mattmireles @john_c_palmer @makinmarkets Ange...
3,1478455370481766400,2022-01-04 19:56:43,@jamie247 I'd like to learn more but first nee...
4,1478445682075729923,2022-01-04 19:18:13,@brianjcho @amytongwu Cosign on those and if y...
...,...,...,...
3245,1406296934852161549,2021-06-19 17:04:51,@sriramk Welcome
3246,1406240299442065408,2021-06-19 13:19:48,@fvckrender @Ioannis_AG @Sothebys Congrats
3247,1406226324318945282,2021-06-19 12:24:16,@PatrickWStanley Maybe all the BTC accumulatio...
3248,1405982338123587588,2021-06-18 20:14:45,@JanetWuNews Optional steam release at the cho...


In [30]:
### DRAFT Collection Object
import requests
import pandas as pd

class DataCollection():
  def __init__(self, token, output_columns=["id", "author", "post_date", "content"]):
        self.token = token
        self.output_columns = output_columns
  def make_df(self, input_arr):
    df = pd.DataFrame(self,input_arr, columns=output_columns)
    return df
  def help(self):
    print("This will be a spot to outline the various functions.")

class TwitterCollection(DataCollection):
  def get_tweets(self, screen_name, num):
    #tweets_list = (screen_name=screen_name, count=num)
    req = requests.get('https://api.twitter.com/2/' )
    return tweets
  
class DiscordCollection(DataCollection):

  def get_channel_messages(self, channel):
    self.headers = {"Authorization": f"Bot {self.token}"}
    channel_path = f'https://discord.com/api/channels/{channel}/messages'
    # Optional additional headers
    res = requests.get(channel_path, headers=self.headers).json()
    for x in res:
      print(x)

class KaggleCollection(DataCollection):
  pass

class RedditCollection(DataCollection):
  pass

class SlackCollection(DataCollection):
  pass


In [32]:
testDiscord = DiscordCollection("ODg1OTQ1MDUxMzcxNDM0MDI0.YTuaoA.L_qt2Keeils_ZdsdENnzUehG-Hw")
#print(testDiscord.output_columns)
print(testDiscord.help())

This will be a spot to outline the various functions
None


OBJECT DESIGN NOTES:

Possibilities:

* Object takes in a df that has been formatted elsewhere. Object would focus primarily on methods that manipulate the data into different file formats and sorting methods.
    * Relies on the user to clean the data and make dataframes that conform to a predtermined column format, which is less magical than the next design
    * Follows DRY, SRP, KISS and YAGNI
    * Pseudo-code examples:
        * `weeks_dataset = new DataSet.weeks(df, filter_by="weeks")`
        * `weeks_and_years_dataset = new DataSet.weeks(df, filter_by=['weeks','years'])`

* Include methods for scraping the most common datasets, something like:
    * Object would attempt to parse out a date object by searching for a column/split point marked as "date," "created_at" or similar
    * Relies less on end user
    * Runs the risk of getting really bloated and trying too hard to cover every edge case for every random data source
    * pseudo-code examples:
        * `kaggle_data = new DataSet.kaggle(credentials, dataset_name)` (are Kaggle datasets standardized, or are there a ton of different presentations? I think the latter)
        * `discord_data = new DataSet.discord(credentials, args* kwargs*)`
        * `twittter_data = new DataSet.twitter(< adapt existing function from this notebook>)`

In [16]:
# TYPE ONE:
# This is not exact; it is late at night and mostly an example, I don't remember the exact syntax off the top of my head
class CleanDataSet:
    def __init__(self, df, file_prefix):
        self.input_df = input_df
        self.file_prefix = file_prefix
        
    def parse_by_weeks(self):
      for x in range(52):
        df = input_df[input_df['Date'].dt.isocalendar().week == x]
        print(df.count)
        # Only make folders and files for weeks where there were tweets
        if not df.empty:
          if not os.path.exists(f'week-{x}'):
            os.makedirs(f'week-{x}')
          df.to_csv(f'week-{x}/{file_prefix}.csv')
      return input_df



# TYPE TWO:

class ScrapeDataSet:
    def __init__(self, data_source, file_prefix):
        self.data_source = data_source
        self.file_prefix = file_prefix
        
    def kaggle(credentials, dataset):
      # Fetch dataset by path (see cell )
        

SyntaxError: ignored

In [ ]:
#HELPER FUNCTIONS FOR ENRON DATA PARSING

import email

email_df = pd.read_csv('enron_path') #just change this to the right path and run these two cells

def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [ ]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

In [ ]:
#get topics using empath

#!pip install empath

from empath import Empath
lexicon = Empath()

#sentence to analyze
content = emails_df['content'][2]

#analyze
nlp = lexicon.analyze(content, normalize=True)

#print the content
print(content)
#print topics with scores greater than 0.5 
for key, v in nlp.items():
    if v > 0.05:
        print(key, v)


In [ ]:
#join empath results with emails_df where empath results are greater than 0.5
#subset emails df to make this run faster for testing
emails_df_subset = emails_df[:100]

emails_df_subset['empath'] = ""
for i, row in emails_df_subset.iterrows():
    content = row['content']
    nlp = lexicon.analyze(content, normalize=True)
    for key, v in nlp.items():
        if v > 0.05:
            emails_df_subset.at[i, 'empath'] = key

emails_df_subset['empath'].head()

In [ ]:
#train gpt3 classifier on emapth topcis
import openai
import config
#for gpt2 if needed
#import gpt_2_simple as gpt2



openai.api_key = config.openai_key

#create new df for training
train_df = emails_df_subset[['content', 'empath']]
#make sure no null values
train_df = train_df[train_df['empath'] != ""]
#im getting errors about the length of the content examples so I need to only select ones that are less than 500 characters till i figure out how to fix this
train_df = train_df[train_df['content'].str.len() < 30]

#create a dictionary of examples
examples_dict = dict(zip(train_df.content, train_df.empath))
#for the sake of this test we will reduce the dict down to the first 3 items or gpt3 will error -- will need to train on a csv for more
examples_dict = {k: examples_dict[k] for k in list(examples_dict)[:3]}
#print(examples_dict)

#turn examples into a list of lists for gpt3 formatting
examples = list(map(list, examples_dict.items()))

#get unique values from the list for the training labels
labels = list(set([x for x in examples_dict.values()]))
#print(labels)

#select random content from emails_df to classify
from random import randrange
num = randrange(200000)

query = emails_df['content'][num]

#remove URLS from query
import re
query = re.sub(r'http\S+', '', query)

#print(query)

def gpt_classify(examples, labels, query):
  response = openai.Classification.create(
        search_model="ada",
        model="curie",
        examples=[examples],
        query=query,
        labels=labels,
      )
  return response.label

test = gpt_classify(examples, labels, query)

print(test)